In [ ]:
#Import libaries
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, ifft
from scipy.io import wavfile 
from scipy.signal import filtfilt, butter
from pylab import *
import os
import librosa
import random
import numpy  as np

#Import self-written functions
from HD_Model_NEW import HD_DataLoader as dl

# feature extraction functions

In [ ]:

def split_extract_vibr_features(data, x):
    """ 
    Calculate the statistical features for every x sample points

    return: 
    feature map of vibration features (both in time and frequenct domain) ->
    x_vibr_means,x_vibr_maxs...x_vibr_fft_means,x_vibr_fft_maxs...y...z...
    type: np array
    size: (n_files * n_time_window(fs*secs/x)) * (30 -> 3(x,y,z)*10(5 time domain features + 5 frequency domain features))
    
    """

    # Reshape the data to split them
    reshaped_data = np.reshape(data, (data.shape[0], data.shape[1], -1, x))  
    #calculate fft of data
    reshaped_data_fft = np.abs(np.fft.fft(reshaped_data, axis=3))

    # calculate statistical features of fft data
    # use split function to split the data into 3 directions (x, y, z dirctions)
    vibr_fft_means = np.split(np.mean(reshaped_data_fft, axis=-1), 3, axis=1)
    vibr_fft_maxs = np.split(np.max(reshaped_data_fft, axis=-1), 3, axis=1)
    vibr_fft_mins = np.split(np.min(reshaped_data_fft, axis=-1), 3, axis=1)
    vibr_fft_medians = np.split(np.median(reshaped_data_fft, axis=-1), 3, axis=1)
    vibr_fft_stds = np.split(np.std(reshaped_data_fft, axis=-1), 3, axis=1)

    # use np.squeeze to eliminate the dimension with only one element
    vibr_fft_means = np.squeeze(vibr_fft_means)
    vibr_fft_maxs = np.squeeze(vibr_fft_maxs)
    vibr_fft_mins = np.squeeze(vibr_fft_mins)
    vibr_fft_medians = np.squeeze(vibr_fft_medians)
    vibr_fft_stds = np.squeeze(vibr_fft_stds)
    
    # split data into separate x,y,z directions
    x_vibr_fft_means, y_vibr_fft_means, z_vibr_fft_means = vibr_fft_means
    x_vibr_fft_maxs, y_vibr_fft_maxs, z_vibr_fft_maxs = vibr_fft_maxs
    x_vibr_fft_mins, y_vibr_fft_mins, z_vibr_fft_mins = vibr_fft_mins
    x_vibr_fft_medians, y_vibr_fft_medians, z_vibr_fft_medians = vibr_fft_medians
    x_vibr_fft_stds, y_vibr_fft_stds, z_vibr_fft_stds = vibr_fft_stds


    # calculate statistical features of original data
    vibr_means = np.split(np.mean(reshaped_data, axis=-1), 3, axis=1)
    vibr_maxs = np.split(np.max(reshaped_data, axis=-1), 3, axis=1)
    vibr_mins = np.split(np.min(reshaped_data, axis=-1), 3, axis=1)
    vibr_medians = np.split(np.median(reshaped_data, axis=-1), 3, axis=1)
    vibr_stds = np.split(np.std(reshaped_data, axis=-1), 3, axis=1)

    # use np.squeeze to eliminate the dimension with only one element
    vibr_means = np.squeeze(vibr_means)
    vibr_maxs = np.squeeze(vibr_maxs)
    vibr_mins = np.squeeze(vibr_mins)
    vibr_medians = np.squeeze(vibr_medians)
    vibr_stds = np.squeeze(vibr_stds)
    
    # split data into separate x,y,z directions
    x_vibr_means, y_vibr_means, z_vibr_means = vibr_means
    x_vibr_maxs, y_vibr_maxs, z_vibr_maxs = vibr_maxs
    x_vibr_mins, y_vibr_mins, z_vibr_mins = vibr_mins
    x_vibr_medians, y_vibr_medians, z_vibr_medians = vibr_medians
    x_vibr_stds, y_vibr_stds, z_vibr_stds = vibr_stds

    # integrade data to a high dimensional feature matrix: size: (n_speeds * n_time_window) * (3(x, y,z) * 10(5 original 
    # features + 5 fft features)); 
    # sequence of features: x_vibr_means,x_vibr_maxs...x_vibr_fft_means,x_vibr_fft_maxs...y...z...
    vibr_feats = []
    for i in [x_vibr_means, x_vibr_maxs, x_vibr_mins, x_vibr_medians, x_vibr_stds, 
              x_vibr_fft_means, x_vibr_fft_maxs, x_vibr_fft_mins, x_vibr_fft_medians, x_vibr_fft_stds,
              y_vibr_means, y_vibr_maxs, y_vibr_mins, y_vibr_medians, y_vibr_stds, 
              y_vibr_fft_means, y_vibr_fft_maxs, y_vibr_fft_mins, y_vibr_fft_medians, y_vibr_fft_stds,
              z_vibr_means, z_vibr_maxs, z_vibr_mins, z_vibr_medians, z_vibr_stds, 
              z_vibr_fft_means, z_vibr_fft_maxs, z_vibr_fft_mins, z_vibr_fft_medians, z_vibr_fft_stds]:
        i = i.flatten()
        vibr_feats.append(i)

    vibr_feats = np.array(vibr_feats).T
    
    return vibr_feats



def split_extract_mic_features(data, x):
    """ 
    Calculate the statistical features for every x sample points
    
    return: 
    feature map of microphone features (both in time and frequenct domain) ->
    mic_means,mic_maxs...mic_fft_means,mic_fft_maxs...
    type: np array
    size: (n_files * n_time_window(fs*secs/x)) * (10 -> 5 time domain features + 5 frequency domain features)
    """
    # only keep the left stereo, since the difference of left and right stereos is very small:
    # shape:[11*1*2646000*2]-> shape:[11*1*2646000*2]
    data = data[:, :, :, 0]

    # eliminate dimensions of only one element
    # shape:[11*1*2646000*1]-> shape:[11*2646000]
    data = np.squeeze(data)

    # Reshape the data to split them every x sample points
    # shape:[11*2646000]-> shape:[11*(2646000/x)*x]
    reshaped_data = np.reshape(data, (data.shape[0], -1, x)) 

    #calculate fft of data
    reshaped_data_fft = np.abs(np.fft.fft(reshaped_data, axis=2))

    # calculate statistical features of fft data
    # shape:[11*(2646000/x)*x]-> shape:[11*(2646000/x)]
    mic_fft_means = np.mean(reshaped_data_fft, axis=-1)
    mic_fft_maxs = np.max(reshaped_data_fft, axis=-1)
    mic_fft_mins = np.min(reshaped_data_fft, axis=-1)
    mic_fft_medians = np.median(reshaped_data_fft, axis=-1)
    mic_fft_stds = np.std(reshaped_data_fft, axis=-1)
    


    # calculate statistical features of original data
    # shape:[11*(2646000/x)*x]-> shape:[11*(2646000/x)]
    mic_means = np.mean(reshaped_data, axis=-1)
    mic_maxs = np.max(reshaped_data, axis=-1)
    mic_mins = np.min(reshaped_data, axis=-1)
    mic_medians = np.median(reshaped_data, axis=-1)
    mic_stds = np.std(reshaped_data, axis=-1)

    # integrade data to a high dimensional feature matrix: size: (n_speeds * n_time_window) * (10(5 original 
    # features + 5 fft features)); 
    # sequence of features: mic_means,mic_maxs...mic_fft_means,mic_fft_maxs...
    mic_feats = []
    for i in [mic_means, mic_maxs, mic_mins, mic_medians, mic_stds, 
              mic_fft_means, mic_fft_maxs, mic_fft_mins, mic_fft_medians, mic_fft_stds]:
        i = i.flatten()
        mic_feats.append(i)

    mic_feats = np.array(mic_feats).T
    
    return mic_feats

# load in data

In [ ]:
HD_dl = dl.HD(r'E:\5ARIP0_Interdisciplinary_team_project\data_analyse\5ARIP10\HD_Model_NEW\HD_data')
HD_database = HD_dl.generate_database()

In [ ]:
# load vibration data

anom_vibr = []
norm_vibr = []

keys_vibr = ['X_vibr', 'Y_vibr', 'Z_vibr']
for test in HD_database:
    if HD_database[test]['attributes']['HD_status'] == 1:  # it is faulty HD
        vibr = {key: HD_database[test]['Vibration'][key] for key in keys_vibr}
        anom_vibr.append([vibr[key] for key in keys_vibr])
    elif HD_database[test]['attributes']['HD_status'] == 0:  # it is working HD
        vibr = {key: HD_database[test]['Vibration'][key] for key in keys_vibr}
        norm_vibr.append([vibr[key] for key in keys_vibr])

anom_vibr = np.array(anom_vibr)
norm_vibr = np.array(norm_vibr)
vibr_data = np.concatenate((norm_vibr, anom_vibr), axis=0)

print(vibr_data.shape)



# load microphone data

anom_mic = []
norm_mic = []

keys_mic = ['Data']
for test in HD_database:
    if HD_database[test]['attributes']['HD_status'] == 1:  # it is faulty HD
        mic = {key: HD_database[test]['Microphone'][key] for key in keys_mic}
        anom_mic.append([mic[key] for key in keys_mic])
    elif HD_database[test]['attributes']['HD_status'] == 0:  # it is working HD
        mic = {key: HD_database[test]['Microphone'][key] for key in keys_mic}
        norm_mic.append([mic[key] for key in keys_mic])

anom_mic = np.array(anom_mic)
norm_mic = np.array(norm_mic)
mic_data = np.concatenate((norm_mic, anom_mic), axis=0)

print(mic_data.shape)

# extract features

In [ ]:
# extract features

vibr_feats = split_extract_vibr_features(data = vibr_data, x = 3200)   # use fs as x
# print(vibr_feats.shape)
mic_feats = split_extract_mic_features(data = mic_data, x = 44100)   # use fs as x
# print(mic_feats.shape)
feature_map = np.hstack((vibr_feats, mic_feats))
# print(feature_map.shape)

# normalize data (use min-max normalizaton: (x-min)/(max-min))
for i in range(feature_map.shape[1]):
    colomn_min = np.min(feature_map[:, i])
    colomn_max = np.max(feature_map[:, i])
    for j in range(feature_map.shape[0]):
        feature_map[j, i] = (feature_map[j, i] - colomn_min)/(colomn_max - colomn_min)